In [1]:
from langchain_community.document_loaders import ApifyDatasetLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain_community.document_loaders.base import Document
from langchain.schema.vectorstore import VectorStoreRetriever
from langchain.agents.agent_toolkits import create_retriever_tool
from langchain.tools import Tool
from langchain.chat_models import ChatOpenAI
from langchain.agents import create_openai_tools_agent
from langchain.agents.agent import AgentExecutor
from langchain import hub

from decouple import AutoConfig

config = AutoConfig(".")

In [2]:
def create_bro_wiki_retriever() -> VectorStoreRetriever:
    loader = ApifyDatasetLoader(
        dataset_id="ef4x5emShPlFoGFPl",
        dataset_mapping_function=lambda dataset_item: Document(
            page_content=dataset_item["text"], metadata={"source": dataset_item["url"]}
        ),
    )
    vectorstore = VectorstoreIndexCreator().from_loaders([loader])
    return vectorstore.vectorstore.as_retriever()

def get_bro_wiki_tool() -> Tool:
    return create_retriever_tool(
        create_bro_wiki_retriever(),
        "bro_wiki_retriever",
        "Procura e retorna informações sobre bRO (Brazilian Ragnarok Online) na sua wiki.",
    )

def get_kafra_agent() -> AgentExecutor:
    tools = [
        get_bro_wiki_tool(),
    ]
    openai_llm = ChatOpenAI(
        model=config("OPENAI_MODEL"),
        temperature=0,
    )
    prompt = hub.pull("hwchase17/openai-tools-agent")
    agent = create_openai_tools_agent(
        tools=tools,
        llm=openai_llm,
        prompt=prompt
    )
    agent_executor = AgentExecutor(
        agent=agent,
        tools=tools,
        verbose=False,
    )
    return agent_executor

In [3]:
kafra_agent_executor = get_kafra_agent()

/home/guilherme/git/atendimento-kafra/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
/home/guilherme/git/atendimento-kafra/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  war

In [5]:
response = kafra_agent_executor.invoke({"input": "Qual efeitos da bata de orleans?"})
print(response["output"])

Infelizmente, não consegui encontrar informações sobre a "Bata de Orleans" no Brazilian Ragnarok Online (bRO) através da ferramenta de busca da wiki. Parece que pode haver um problema com a ferramenta de busca ou o item pode não estar listado na wiki sob esse nome.

Se você estiver procurando pelos efeitos da "Bata de Orleans" no jogo, posso fornecer informações com base no conhecimento geral sobre o item em outras versões do Ragnarok Online, mas tenha em mente que pode haver diferenças específicas no bRO. A "Bata de Orleans" (Orleans's Gown) é geralmente conhecida por oferecer uma redução no tempo de conjuração variável de habilidades mágicas. Se você precisar de informações precisas e atualizadas específicas para o bRO, recomendo verificar diretamente no jogo, se possível, ou em fóruns e comunidades de jogadores do bRO.
